In [105]:
from clrs._src.algorithms.graphs import bfs

In [106]:
import numpy as np
from torch_geometric.data import Data
import networkx as nx

In [107]:
class CLRSData(Data):
    """A data object for CLRS data."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [108]:
import numpy as np
import networkx as nx
from clrs._src.algorithms.graphs import bfs
import matplotlib.pyplot as plt
import torch
class graph_generator:
    def __init__(self, n, p, directed=False, type='erdos_renyi'):
        self.n = n # number of nodes
        self.p = p # probability of edge creation
        self.s = np.random.randint(0, n) # source node
        if type == 'erdos_renyi':
            self.graph = nx.erdos_renyi_graph(n, p, directed=directed)
        self.adj = nx.to_numpy_array(self.graph)
        self.edges_indexes = self.get_edges_indexes(self.adj)
        # write edgelist in edge_indexes
        #self.edges_indexes = nx.to_edgelist(self.graph)
        # make it into a list of lists 2x1 for the edge indexes
        #self.edges_indexes = [[x[0] for x in self.edges_indexes], [x[1] for x in self.edges_indexes]]
    
        self.pi, probes = bfs(self.adj, self.s)
        self.pi_h = probes['hint']['node']['pi_h']['data']
        self.reach_h = probes['hint']['node']['reach_h']['data']

        self.pi = self.get_edges(self.adj, self.pi)
        self.pi_h = np.array([self.get_edges(self.adj, x) for x in self.pi_h])
        self.pos = np.arange(0, n)/n
        self.length = (self.pi_h).shape[0] # length of the sequence of hints

        dict = {'edge_index': self.edges_indexes, 'pos': self.pos, 'length': self.length, 's': self.s, 'pi': self.pi, 'reach_h': self.reach_h, 'pi_h': self.pi_h}
        dict = {k: self.to_torch(v) for k,v in dict.items()}
        dict['hints'] = np.array(['reach_h', 'pi_h'])
        dict['inputs'] = np.array(['pos', 's'])
        dict['outputs'] = np.array(['pi'])
        self.tensor = CLRSData(**dict)

    def get_edges_indexes(self, A):
        edge_indexes = [[],[]]
        """Create a 2x1 list of lists to store the indexes of the edges in the adjacency matrix."""
        for i in range(len(A)):
            for j in range(len(A)):
                if A[i][j] == 1:
                    edge_indexes[0].append(i)
                    edge_indexes[1].append(j)
        return edge_indexes
    
    def get_edges(self, A, pi):
        """Returns a binary list of edges in the graph. If the edge was accessed by the algorithm, it is marked as 1, otherwise it is 0."""
        edge_indexes = self.get_edges_indexes(A)
        edges = np.zeros(len(edge_indexes[0]))
        for i in range(len(pi)):
            if pi[i] != -1:
                for j in range(len(edge_indexes[0])):
                    if edge_indexes[1][j] == i and edge_indexes[0][j] == pi[i]:
                        edges[j] = 1
        return edges
    
    def draw(self):
        nx.draw(self.graph, with_labels=True)
        plt.show()
    
    def to_torch(self, value):
        if isinstance(value, np.ndarray):
            return torch.from_numpy(value)
        elif isinstance(value, torch.Tensor):
            return value
        else:
            return torch.tensor(value)
        
    def save(self, path):
        torch.save(self.tensor, path)

    def saveRawFile(self, path):
        """saves the graph as an edge list"""
        nx.write_edgelist(self.graph, path, delimiter='\t', data=False)

In [109]:
# test de la clase
n =10
p = 0.3
g = graph_generator(n, p)

In [122]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os
import os.path as osp

class GraphGenerator:
    def __init__(self, n, p, root:str="./data", num_graphs:int=100, directed:bool=False, max_workers:int=2):
        self.n = np.array(n)
        self.p = np.array(p)
        self.directed = directed
        self.path = root
        self.max_workers = max_workers
        self.num_graphs = num_graphs

        # check if the directory exists
        if not os.path.exists(osp.join(self.path)):
            os.makedirs(osp.join(self.path))
        if not os.path.exists(osp.join(self.path, 'raw')):
            os.makedirs(osp.join(self.path, 'raw'))
    
    def generate(self):
        print('Generating ' + str(self.num_graphs) + ' graphs')
        with tqdm(total=self.num_graphs) as pbar:
            with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                for i in range(self.num_graphs):
                    executor.submit(self.generate_graph(i))
                    pbar.update(1)

    def generate_graph(self, i):
        g = graph_generator(n, p, directed=self.directed)
        g.saveRawFile(osp.join(self.path, 'raw', 'er_graph_' + str(i) + '.edgelist'))

In [137]:
from torch_geometric.data import Dataset
import os.path as osp
import glob
import torch
from torch_geometric.data import Dataset, download_url
class RandomGraphDataset(Dataset):
    def __init__(self, root: str = "./data", gen_num_graph: int = 100, n: int = 10, p: float = 0.01, directed: bool = False, transform=None, pre_transform=None):
        self.n = n
        self.p = p
        self.directed = directed
        self.gen_num_graphs = gen_num_graph
        super(RandomGraphDataset, self).__init__(root, transform, pre_transform)
    
    @property
    def raw_file_names(self):
        return list(map(os.path.basename,glob.glob(osp.join(self.raw_dir, "*.edgelist"))))
    
    @property
    def processed_file_names(self):
        return list(map(os.path.basename,glob.glob(osp.join(self.processed_dir, "*.pt"))))
    
    def download(self):
        """Creates an instance of the graph generator and generates the graphs."""
        gen = GraphGenerator(n=self.n, p=self.p, root=self.root, num_graphs=self.gen_num_graphs, directed=self.directed) 
        gen.generate()
           
    def process(self):
        if not os.path.exists(osp.join(self.root, 'processed')):
            os.makedirs(osp.join(self.root, 'processed'))
        # for each raw file, create a graph, apply bfs and create a tensor with the data
        for i in range(self.gen_num_graphs):
            # go through the raw files and create a graph without the need to generate it again
            g = nx.read_edgelist(osp.join(self.root, 'raw', 'er_graph_' + str(i) + '.edgelist'), nodetype=int)
            adj = nx.to_numpy_array(g)
            edges_indexes = self.get_edges_indexes(adj)
            s = np.random.randint(0, len(adj))
            pi, probes = bfs(adj, s)
            pi_h = probes['hint']['node']['pi_h']['data']
            reach_h = probes['hint']['node']['reach_h']['data']
            pi = self.get_edges(adj, pi)
            pi_h = np.array([self.get_edges(adj, x) for x in pi_h])
            pos = np.arange(0, len(adj))/len(adj)
            length = (pi_h).shape[0]
            dict = {'edge_index': edges_indexes, 'pos': pos, 'length': length, 's': s, 'pi': pi, 'reach_h': reach_h, 'pi_h': pi_h}
            dict = {k: self.to_torch(v) for k,v in dict.items()}
            dict['hints'] = np.array(['reach_h', 'pi_h'])
            dict['inputs'] = np.array(['pos', 's'])
            dict['outputs'] = np.array(['pi'])
            tensor = CLRSData(**dict)

            if self.pre_transform is not None:
                tensor = self.pre_transform(tensor)
            torch.save(tensor, osp.join(self.root, 'processed', 'data' + str(i) + '.pt'))
        
    def len(self):
        """Returns the number of files."""
        return len(self.raw_file_names)
    
    def get(self, idx):
        """Returns the idx-th graph in the dataset. (Tensor)"""
        data = torch.load(osp.join(self.root, 'processed', 'data' + str(idx) + '.pt'))
        return data
        
    def get_edges_indexes(self, A):
            edge_indexes = [[],[]]
            """Create a 2x1 list of lists to store the indexes of the edges in the adjacency matrix."""
            for i in range(len(A)):
                for j in range(len(A)):
                    if A[i][j] == 1:
                        edge_indexes[0].append(i)
                        edge_indexes[1].append(j)
            return edge_indexes
    
    def get_edges(self, A, pi):
        """Returns a binary list of edges in the graph. If the edge was accessed by the algorithm, it is marked as 1, otherwise it is 0."""
        edge_indexes = self.get_edges_indexes(A)
        edges = np.zeros(len(edge_indexes[0]))
        for i in range(len(pi)):
            if pi[i] != -1:
                for j in range(len(edge_indexes[0])):
                    if edge_indexes[1][j] == i and edge_indexes[0][j] == pi[i]:
                        edges[j] = 1
        return edges

    def to_torch(self, value):
        """Transforms a numpy array into a torch tensor."""
        if isinstance(value, np.ndarray):
            return torch.from_numpy(value)
        elif isinstance(value, torch.Tensor):
            return value
        else:
            return torch.tensor(value)

In [141]:
# test de la clase
n = 10
p = 0.3
dataset = RandomGraphDataset(root='./data', gen_num_graph=10, n=n, p=p)

Generating 10 graphs


100%|██████████| 10/10 [00:00<00:00, 430.57it/s]
Processing...
Done!


In [142]:
# check one of the tensors content
dataset.get(0)

CLRSData(edge_index=[2, 30], pos=[10], length=3, s=3, pi=[30], reach_h=[3, 10], pi_h=[3, 30], hints=[2], inputs=[2], outputs=[1])

In [29]:
import os
n = [10, 20]
p = [0.3, 0.5]
path = os.getcwd() + '/data'

In [30]:
num_graphs = 10
g = GraphGenerator(n, p, path)
g.generate(num_graphs)

Generating 10 graphs


100%|██████████| 10/10 [00:00<00:00, 331.80it/s]


In [31]:
# access the first graph
graph = torch.load(path + '/graph0.pt')
print(graph)

CLRSData(edge_index=[2, 52], pos=[11], length=3, s=7, pi=[52], reach_h=[3, 11], pi_h=[3, 52], hints=[2], inputs=[2], outputs=[1])


In [10]:
graph.edge_index

tensor([[ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,
          3,  3,  4,  4,  4,  4,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  7,  8,
          8,  8,  8,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10],
        [ 1,  2,  4,  6, 10,  0,  2,  5,  8,  9,  0,  1,  3,  7, 10,  2,  4,  6,
          7, 10,  0,  3,  6,  9,  1,  9,  0,  3,  4, 10,  2,  3,  8,  9, 10,  1,
          7,  9, 10,  1,  4,  5,  7,  8, 10,  0,  2,  3,  6,  7,  8,  9]])

In [11]:
# create an encoder class
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim = 128):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.lin = nn.Linear(input_dim, hidden_dim)

    def forward(self, x):
        return self.lin(x)